# Spectral Clustering

The spectrum of a matrix refers to the eigenvalues of the matrix. In spectral clustering the eigenvalues of a similarity matrix is used to perform dimensionality reduction, prior to clustering. In this tutorial we will take the reader through the construction of a similarity matrix, the construction of a Laplacian matrix, the reduction of the laplacian to its eigenvalues and how these eigenvalues can be used to cluster the graph.

### Similarity and Degree is used to construct Laplacian

Consider the case where we have two locations on a map $l_i, l_j$ $\in L$, we can define a distance measure $d$ such that $\forall l_i,l_j \in L: d(l_i,l_j) \mapsto {1 \cup 0}$, dependent on whether a road directly connects $l_i$ to $l_j$.

Once a distance measure has been defined a similarity matrix can be constructed as a symmetric matrix $A$, where $A_{ij} \geq 0$ represents whether $i$ and $j$ are neighboring towns.

The general idea of spectral clustering is to use a regular clustering algorihtm (such as k-means) on relevant eigenvectors of a Laplacian Matrix of $A$. The question to answer next is how do we construct the Laplacian?

Once we have a similarity matrix $A$ we can construct a distance matrix $D$, such that $D_{ii} = \sum_j A_{ij}$. This produces a diagonal matrix the elements of which for a given town $i$, represents the number of towns $j$ that neighbor $i$.

There are many different types of Laplacian which a few of which we shall delve deeper into later in this article. For now a __simple Laplacian__ matrix, can be constructed using the formula 

$$L = D - A$$

Elements $L_{ii}$ represent the number of neighboring towns, and elements $L_{ij}: i\neq j$ specify whether there is road between town $i$ and town $j$.

This leaves us with our Laplacian matrix.

#### Symmetric normalised Laplacian

The symmetric normalised Laplacian is defined as 

$$L_{sym} = D^{-\frac{1}{2}}LD^{-\frac{1}{2}} = I - D^{-\frac{1}{2}}AD^{-\frac{1}{2}}$$

Wher $L$ is the unnormalised Laplacian, $A$ is the adjacency matrix and $D$ is the degree matrix. The reciprocal square root of $D$, $D^-{\frac{1}{2}}$ is a diagonal matrix who's entries are the positive square roots of the diagonal entries $D$. 

Once one has $L^{sym} = SS^*$, where $S$ is the matrix with row and columns corresponding to vertices and edges repectively. Such that each column corresponding to an edge $e= \{u, v\}$ has an entry $\frac{1}{\sqrt{d_u}}$ in the row corresponding to $u$, an entry $\frac{1}{\sqrt{d_v}}$ in the row corresponding to $v$, and $0$ elsewhere.

The eigenvalues of the normalised Laplacian are non-negative, and satisfy $0 = \lambda_0 \leq \dots \leq \lambda_{n-1} \leq 2$. These eigenvalues known as the spectrum, relate well to other graph invariants for general graphs.

#### Random Walk Normalised Laplacian
The random walk normalised Laplacian is defined as

$$L^{rw} = D^{-1}L$$

In this case $D^{-1}$ is simply the a diagonal matrix with entries which reciprocals of the positive diagonal entries of $D$.

For isolated vertices a common choice is to set the corresponding element $L^{rw}_{i,i}$ to $0$. This leads to the nice property that the multiplicity of the eigenvalue $0$ is equal tothe numebr of connected components in the graph.



### Eigenvectors and Eigenvalues

Once the Laplacian matrix has been constructed the eigenvalues can be calculated. Let $\{\lambda_0, \dots, \lambda_n\}$ be an ordered set of eigenvalues such that $\lambda_0 \leq \lambda_1 \leq \dots \leq \lambda_n$. As every row and column of the matrix $L$ sum to $0$, $\lambda_0 = 0$ because the vector $\vec{v_0}=(1,1,\dots,1)$. 

The second smallest eigenvalue $\lambda_1$ is called the \textbf{Fiedler} value. The Fiedler value is only $> 0$ if L represents a fully connected graph. The corresponding vector called the Fiedler Vector which approximates the sparsest cut on the graph $L$. Therfore this  

Clustering based on the Fiedler vector means we are clustering on a line in the $\mathbb{R}$ space. This means we have a number of options available to define the clusters, a common choice is k-means.


In [11]:
from sklearn.datasets import make_circles
from sklearn.neighbors import kneighbors_graph
import numpy as np

X = [[1,2], [2,3], [2,1], [30,33], [32,30], [31,31]]

A = kneighbors_graph(X, n_neighbors=2).toarray()

# create the graph laplacian
D = np.diag(A.sum(axis=1))
L = D-A
print("A:\n", A)
print("D:\n", D)
print("L:\n", L)

# find the eigenvalues and eigenvectors
vals, vecs = np.linalg.eig(L)
idx = vals.argsort()[::1]
vals = vals[idx]
vecs = vecs[:,idx]

print("Sort order: ", order)
print("Vals:\n", vals)
print("Vecs:\n", vecs)

# use Fiedler value to fi
clusters = vecs[:,1] < 0
print("Fiedler Vector:\n", vecs[:,1])
print("Clusters:\n", clusters)                 

A:
 [[0. 1. 1. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 1. 1. 0.]]
D:
 [[2. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 2.]]
L:
 [[ 2. -1. -1.  0.  0.  0.]
 [-1.  2. -1.  0.  0.  0.]
 [-1. -1.  2.  0.  0.  0.]
 [ 0.  0.  0.  2. -1. -1.]
 [ 0.  0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1. -1.  2.]]
Sort order:  [1 4 0 2 3 5]
Vals:
 [-1.11022302e-16 -1.11022302e-16  3.00000000e+00  3.00000000e+00
  3.00000000e+00  3.00000000e+00]
Vecs:
 [[-0.57735027  0.          0.81649658  0.30959441  0.          0.        ]
 [-0.57735027  0.         -0.40824829 -0.80910101  0.          0.        ]
 [-0.57735027  0.         -0.40824829  0.49950661  0.          0.        ]
 [ 0.         -0.57735027  0.          0.          0.81649658  0.30959441]
 [ 0.         -0.57735027  0.          0.         -0.40824829 -0.80910101]
 [ 0.         -0.57735027  0.          0.         -0.408